In [4]:
import numpy as np
from copy import deepcopy
from math import ceil, floor

class Bqf:
    def __init__(self, a, b, c):
        self.terms = [["x^2", a, lambda x, y: x^2],
                      ["xy", b, lambda x, y: x * y],
                      ["y^2", c, lambda x, y: y]]
        
        self.update_vals(a,b,c)
        
        self.mat = [
            [self.a, self.b / 2],
            [self.b / 2, c]
        ]
        
        self.det  = self.determinant()
        self.disc = self.discriminant()
        
        if (self.disc == 0): self.type = "Perfect square"
            
        elif (self.disc < 0):
            if (self.a > 0): self.type = "Positive definite"
            else:            self.type = "Negative definite"
                
        else:                self.type = "Indefinite"
        
        self.transform = []
        
    def __call__(self, x, y):
        return sum([i[1] * i[2](x,y) for i in self.terms])
    
    def __repr__(self):
        out = ""
        for i in range(len(self.terms)):
            if self.terms[i][1] != 0:
                
                if i > 0:
                    if self.terms[i][1] > 0:
                        out += "+ "
                    else:
                        out += "- "
                elif (self.terms[i][1] < 0):
                    out += "-"
                        
                if abs(self.terms[i][1]) != 1:
                    out += str(abs(self.terms[i][1]))
                    
                out += self.terms[i][0] + " "

        return out[:-1]
    
    def __eq__(self, other):
        return np.allclose(self.reduce().mat, other.reduce().mat)
    
    def determinant(self):
        return (self.mat[0][0] * self.mat[1][1]) - (self.mat[0][1] * self.mat[1][0])
    
    def discriminant(self):
        return self.determinant() * -4    
    
    def is_reduced(self):
        return -abs(self.a) < self.b <= abs(self.a) < abs(self.c) or\
                0 < self.b -<= abs(self.a) == abs(self.c)
    
    def update_vals(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c
        
        self.terms[0][1] = a
        self.terms[1][1] = b
        self.terms[2][1] = c
        
    def reduce(self):
        if (self.is_reduced()):
            print("already reduced")
            return self
        
        red = deepcopy(self)
        
        while (not red.is_reduced()):
            print(red.mat)
            
            if (abs(red.a) > abs(red.c)):
                red.mat = np.matmul(red.mat, [[0, 1],[-1, 0]])
                print([[0, -1],[1, 0]])
            else:
                m = 0
                # find transformation matrix
                if (red.a > 0):
                    m = floor((abs(red.a) - red.b)/(2*red.a))
                else:
                    m = ceil((abs(red.a) - red.b)/(2*red.a))

                red.mat = np.matmul(red.mat, [[1, m],[0, 1]])
                print([[1, m],[0, 1]])
                        
            #assert red.mat[0][1] * 2 == red.b
            
            if (abs(red.a) == abs(red.c) and red.b < 0):
                red.mat = np.matmul(red.mat, [[0, 1],[-1,0]])
            
                red.update_vals(red.mat[0][0], red.mat[0][1] * 2, red.mat[1][1])
                    
        print(red.mat)
        print("is reduced")
        assert red.mat[0][1] * 2 == red.b
        print(red.mat[0][1])
        print(red)
        return red

SyntaxError: invalid syntax (<ipython-input-4-ade6b830675a>, line 65)

### 3.6
#### 4.) Use the method of Example 3 to find integers x and y such that $x^2+y^2=89753$, given that this number is prime.

In [8]:
# find bqf with determinant -4 and f(1,0) = target number

def pgen_bqf(x: int):
    # verify primality
    assert x % 
    return Bqf()

### 9.1

#### 1.) If $f(x)|g(x)$ and $g(x)|f(x)$, prove that there is a rational number c such that $g(x) = cf(x)$. 

$f(x)|g(x) \implies \exists u_1(x): f(x)u_1(x)=g(x)$.

Likewise, $g(x)|f(x) \implies \exists u_2(x): g(x)u_2(x)=f(x)$.

Using substitution, we can form the following equation:

$f(x)=f(x)u_1(x)u_2(x) \implies u_1(x)u_2(x)=1 \implies u_1(x)=u_2(x)^{-1}$

Note that if $u_2(x)$ was not a rational number, its inverse would not be in $\mathbb{Q}[x]$, as one of its terms would be of the form $x^{-i}$ for some positive integer i.

Therefore, $u_2(x)$ must be a rational number, which implies that $u_1(x) = u_2(x)^{-1}$ is a rational number.

If we set $c = u_1(x)$, $g(x)=cf(x)$.

#### 2.) If $f(x)|g(x)$ and $g(x)|h(x)$, prove that $f(x)|h(x)$.

$f(x)|g(x) \implies \exists u_1(x): f(x)u_1(x)=g(x)$.

$g(x)|h(x) \implies \exists u_2(x): g(x)u_2(x)=h(x)$.

$\implies f(x)u_1(x)u_2(x)=h(x)$ by substitution.

Let $u_3(x)=u_1(x)u_2(x)$, so $f(x)u_3(x)=h(x)$.

Therefore, $f(x)|h(x)$.

#### 3.) If $p(x)$ is irreducible and $g(x)|p(x)$, prove that either $g(x)$ is a constant or $g(x) = cp(x)$ for some rational number $c$. 

Because $p(x)$ is irreducable, it cannot be factored.

However, observe that any polynomail in $\mathbb{Q}[x]$ can be divided by some constant in $\mathbb{Q}$ without leaving $\mathbb{Q}[x]$.

Additionally, if $g(x)$ is some constant multiple of $p(x)$, $\frac{p(x)}{g(x)}=c$ where $c$ is some rational number.

Any other case, however, counts as factoring, so the above two cases are the only possible for $g(x)|p(x)$.

#### 5.) If a polynomial $f(x)$ with integral coefficients factors into a product $g(x)h(x)$ of two polynomials with coefficients in $\mathbb{Q}$, prove that there is a factoring $g_1(x)h_1(x)$ with integral coefficients. 

$f(x)$ can be expressed as a multiple of the an equivalent primitive polynomial; a primitive polynomial multiplied by the gcd of the coefficients. We will call this $cp(x)$.

By Gauss's Lemma, $p(x)=g(x)h(x)$ implies that both $g(x)$ and $h(x)$ are primitive.

#### 6.)  If $f(x)$ and $g(x)$ are primitive polynomials, and if $f(x)|g(x)$ and $g(x)|f(x)$, prove that $f(x) = \pm g(x)$. 

As proved in problem 9.1.1, $\exists c: g(x) = c f(x)$.

Let us assume that $c$ is not $\pm 1$.

Because $c|g(x)$, we've found a contradiction to $f(x)$'s primitivity, as a primitive polynomial must also be irreducable, but $g(x)$ is a nontrivial polynomial which divides $f(x)$. 

Therefore, $c$ must be $\pm 1$.

Format for straight edge and compass construction?